# Indeed Scrapper

### Indeed Company Reviews Scrapper

The way the links work in this website makes it easier to scrape the data as it loops by 20 steps
so it's easy to set it up in a for range and counting 20 steps and there's no need to use for example clicks
and the data is already all visible in the html 
The link is also dynamic and allows to exchange the companies via the link so the company was made as variable


The website reviews are stored in a general container, if we try to scrape directly  the instead of going
inside of the container the we're getting empty values

In [29]:
#Import Necessary Librabries
import time
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
import os,glob

In [30]:
#Creating a DataFrame to store the data 
extracted_reviews = pd.DataFrame({'review title': pd.Series([], dtype='string'),
                                    'review':pd.Series([], dtype='string'),
                                    'pros':pd.Series([], dtype='string'),
                                    'cons':pd.Series([], dtype='string'),
                                    'date':pd.Series([], dtype='string'),
                                    'job':pd.Series([], dtype='string'),
                                    'location':pd.Series([], dtype='string'),
                                    'rating':pd.Series([], dtype='float')})

In [31]:
#Adding the argument to make it not visible
options = Options()
options.add_argument('--headless')
#options.add_argument('--disable-gpu')  # this may be necessary in windows/chrome
options.add_argument("--disable-blink-features") # avoid Robot detection
options.add_argument("--disable-blink-features=AutomationControlled")

In [32]:
#Open Chrome with Selenium
browser = webdriver.Chrome('E:\Documentos\chromedriver.exe', options = options)  # Optional argument, if not specified will search path.


# Open Firefox with Selenium (unmark to run with firefox)
#binary = FirefoxBinary('/Applications/Firefox.app/Contents/MacOS/firefox-bin')  # replace the Firexfox path with the one on your computer
#browser = webdriver.Firefox(firefox_binary=binary, options=options)
#browser = webdriver.Firefox(options=options)

In [33]:
# Add a list and variable to keep track of progress and missing files
missing_files = []

run = 0

In [34]:
#Select the path where the files are stored
folder_path = 'E:\Documentos\Pages\wqw'

#Iterate through the files in the given folder
for filename in glob.glob(os.path.join(folder_path, '*.html')):
    time.sleep(5)
    browser.get(filename)
    html_source = browser.page_source
    soup = BeautifulSoup(html_source,"html.parser")
    #print(soup)
    browser.quit()
    
    try:
        # Reviews are divided by containers and not all of them have the same attributes for we're scrapping per container
        data = soup.find('div',{"id": 'cmp-container'})
        reviews = data.find_all(class_ = "cmp-Review-container")
        for x in reviews:
             #Pull the title
            title = x.find(attrs = {'class':'cmp-Review-title'})
            
            #Pull the review content
            review = x.find('div', {'class': 'cmp-Review-text'})
            
            #Not all reviews have the pros and cons (but if one exists both do)
            try:
                pros = x.find(attrs = {'class':'cmp-ReviewProsCons-prosText'}).find(attrs = {'class': 'cmp-NewLineToBr-text'}).get_text()
                cons = x.find(attrs = {'class':'cmp-ReviewProsCons-consText'}).find(attrs = {'class': 'cmp-NewLineToBr-text'}).get_text()
            except:
                pros,cons = None,None
                
            # Pull the Review Date
            rev_date = x.find(attrs = {'class':'cmp-Review-author'})
            
            # Pull the Job title(not always available)
            try:
                job = x.find('a', {'class':'cmp-ReviewAuthor-link'}).text
            except:
                job = None
            
            #Pull the location
            location = x.find(attrs = {'class':'cmp-Review-author'})
            
            # Pull the rating given by the reviewer
            rating = x.find(attrs = {'class':'cmp-ReviewRating-text'})
            
            # Save the data to the DataFrame and make the necessary changes to the text
            extracted_reviews = extracted_reviews.append({'review title': title.text,
                                                      'review': review.text,
                                                      'pros': pros,
                                                      'cons': cons,
                                                      'date': rev_date.text.split("-")[-1],
                                                      'job': job,
                                                      'location': location.text.split("-")[-2],
                                                      'rating': rating.text
                                                      }, ignore_index=True)
            print(run)
            run += 1
    except:
        missing_files.append(filename)
        continue
        

#Convert the reviews as float 
extracted_reviews['rating']= [x.replace(",",".") for x in extracted_reviews['rating']]


#Save the reviews   
extracted_reviews.to_excel("Mcdonald's Reviews.xlsx")  

print("Reviews Saved!")  

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
Reviews Saved!


In [35]:
print(missing_files) 

[]


In [36]:
extracted_reviews

,review title,review,pros,cons,date,job,location,rating
0,Customers are disgustingly rude,I’ve been working at McDonald’s for a year and...,"Free meals, benefits, pay, flexibility","Long hours, exhausting, horrible treatment fro...","May 14, 2021",McDonalds Crew Member,Telford,3.0
1,Stressful and confusing,They don't let you work in one station. They m...,"Free lunch only be able to get one burger, onl...","Long hours, comfusing managers, confusing roles.","January 5, 2021",CUSTOMER SERVICE,North London,2.0
2,Good company,I enjoyed working here for the time that I did...,Free lunches and discount,Low pay,"January 5, 2021",Crew Member - Part Time,Sheffield,4.0
3,Meh,Fellow crew members were nice the same for slm...,Free food on shift.,Management.,"January 4, 2021",Crew Member,Wellington,2.0
4,Fun workplace,"I worked here for 4 years, I got promoted to c...",Free lunches,Stressful/tiring,"January 3, 2021",Shift Manager,Norwich,4.0
5,Don’t work there,"You get no respect and treated really bad, wou...",None,None,"January 2, 2021",Customer Care Assistant,"Swansea, Swansea",1.0
6,Horrible,"If I could give 0 stars I would, staff are rud...",Nothing,Everything,"December 30, 2020",CREW MEMBER,Yardley,1.0
7,Okay place to work for first job,With the past 3 years working there I used to ...,None,None,"December 30, 2020",Crew member McDonalds-full time,"Nottingham, Nottinghamshire",1.0
8,Bad training,When I was working here I had training for the...,Free lunches,Bad training,"December 30, 2020",Team Member,"Mare, Somerset",3.0
9,Fun place to work,"Was a fun place to work most of the time, met ...",Free food on breaks,"Franchisee, not much discount, unfair progression","December 30, 2020",Crew Trainer,Tyldesley,3.0
